# Template for Creating Sacrament Meeting Programs

## Uses MarkupPy to generate the HTML

In [17]:
#
# Sugar Land 2nd Ward HTML Program Writer
#
# Pete Slater
# June 2022

import hymndict # Titles and links to hymns
import htmlpy # Local copy of source code
from htmlpy import oneliner as e
import artlinks # Links to artwork
import sys
import datetime as dt

# Customized functions to create the HTML for standard agenda items

In [18]:
'''
Define functions for formatting each of the items that can appear in a program.
Build the program in the main script by calling the functions.

'''
def officer(role, officer, page):
    page.p(e.b(role+":")+" "+officer,align="center")

def pagetitle(unit, meeting, meetdate,page):
    page.h1(unit, align="center")
    page.h2(meeting, align="center")
    page.h2(meetdate, align="center")

def speaker(name, page):
    page.h2("Speaker", align="center")
    page.p(name,align="center")
    
def testimonies(page):
    page.h2("Bearing of Testimonies", align="center")

def music(number, description, page):
    page.h2(description,align="center")
    [hymntitle, hymnurl] = hymndict.hymns[number]
    page.p(e.a("#" + str(number)+", "+hymntitle, href=hymnurl),align="center")
    
def specialmusic(performers,page,title=None,accompanist=None):
    page.h2("Special Musical Number",align="center")
    if title != None:
        page.p(e.i(title),align="center")
    page.p(performers,align="center")
    if accompanist != None:
        page.p("Acc. by " + accompanist, align="center")
        
def thought(text, author, page):
    page.hr()
    page.p(text, align="center")
    page.p("- "+author,align="center")
 
# Announcements are passed as a list of text strings
def announcements(textlist,page):
    page.hr()
    page.h1("Announcements", align="center")
    for txt in textlist:
        page.p(txt,align="center")

# Custom section for Preparedness Corner, etc.
def custom(title, txtlist, linklist, page):
    page.hr()
    page.h1(title, align="center")
    if txtlist != None:
        for txt in txtlist:
            page.p(txt,align="center")
    if linklist != None:
        for link in linklist:
            page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Place some links, passed as a list containing text and url
def links(linklist, page):
    page.hr()
    page.h1("Links", align="center")
    for link in linklist:
        page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Print credits at the bottom of the page
def credit(editor, page):
#'''Print credits and datestamp ''' 
    today = dt.date.today()
    page.hr()
    text = today.strftime("Edited %B %d, %Y by ") + editor
    page.small(text,align="center")        
    

## Functions for the cleaning calendar

In [19]:
def nextsunday(n):
#""" Return next n Sundays, including today if it's Sunday"""
 today = dt.date.today()
 daynum = today.weekday() # Sunday is day #6

 if daynum == 6:
  basedate = today 
 else:
  basedate = today + dt.timedelta(6-daynum)
    
 sundays = []
 for i in range(n):
  nxtSunday = basedate + dt.timedelta(i*7)
  sundays.append(nxtSunday.strftime("%B %d, %Y"))
 return sundays

def nextsaturday(n):
#""" Return next n Saturdays, including today if it's Saturday"""
 today = dt.date.today()
 daynum = today.weekday() # Saturday is day #5

 if daynum == 5:
  basedate = today 
 else:
  basedate = today + dt.timedelta(5-daynum)
    
 saturdays = []
 for i in range(n):
  nxtSaturday = basedate + dt.timedelta(i*7)
  saturdays.append(nxtSaturday.strftime("%B %d"))
 return saturdays

In [20]:
roster = {
    "June 24" : "Sugar Land 1st",
    "July 01" : "Bradford, Arnold, Emerson, Markwalter",
    "July 08" : "Clark, Lambert, Blackburn, Ganci, Martin, Robinson",
    "July 15" : "Crandall, Blanco, Garcia, Moline, Thibault",
    "July 22" : "Crosby, Boyer, Griffin, Mike & Dawnette Moore, MacKinnon",
    "July 29" : "Maplewood 3rd Branch",
    "August 05" : "Draney, Buchanan, Hettinger, Morton, Tracey",
    "August 12" : "Leavitt, Chen, Hintze, Ozomah, Seegmiller",
    "August 19" : "Powell, Clements, Horowitz, Quam, Wightman",
    "August 26" : "Kaye Reynolds, Debow, Jeffery, Rasmussen, Winningham",
    "September 02" : "Roskelley, Ekstrom, Ocampo, Juarez, Jason & Deann Moore, Slater"
}

# Format the agenda items that are always present
Customize for each week with date and type of meeting, i.e. sacrament, fast and testimony, ward conference

In [21]:
Title = "Sugar Land Second Ward"
Title = "Houston Texas South Stake"
header = "Sacrament Meeting"
#header = "Fast and Testimony Meeting"
#header = "Stake Conference"
footer = ""
styles = ( 'layout.css', 'alt.css', 'images.css' )

page = htmlpy.page( )
page.init()
page.br( )
 
# Make sure it will look good on all devices
page.meta(name="viewport", content="width=device-width, initial-scale=1.0")

# Define the elements on the current week's programs here

pagetitle("Sugar Land Second Ward", "Sacrament Meeting", "July 23rd, 2023", page)

## Artwork
Pick an artwork from the defined list of links in artlinks.py and enter the number here

In [22]:
# Place an artwork from the imported dictionary of links
#page.p(e.img(width=299, height=300*0.8, src=artlinks.art[2]), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src=artlinks.art[7]), align="center")

## Officers

In [23]:
officer("Presiding","President Travis Bird", page)
#officer("Conducting","Bishop Joey Powell", page)
#officer("Presiding","Brother Dale Crosby", page)
#officer("Conducting","Brother Dale Crosby", page)
officer("Conducting","Brother Jared Draney", page)

## Opening and sacrament hymns

In [24]:
music(30,"Opening Hymn",page)
music(176,"Sacrament Hymn", page) 

## Variable part of the program

### If it's fast and testimony meeting, uncomment this section and comment out the sacrament meeting items

In [25]:
# Uncomment this line for fast and testimony meeting
#testimonies(page)

### For a sacrament meeting or ward conference, comment out the testimonies and use these sections as needed

In [26]:
# Use lines like these to form the speaking part of the program

# Special code for the Primary and Christmas Programs
#txtlist = (
#    'Words and Music Celebrating the Birth of Jesus Christ',
#    'Program under the direction of Mindy Draney',
#    'Program written by Becky Slater',
#    'Congregation to join in singing when directed.'
#    )
#linklist = None

#custom("Primary Program", txtlist,linklist, page)
#specialmusic("Primary Children",page, title="Fathers")
#specialmusic("Primary Children",page, title="Daddy's Homecoming")

speaker("Asia Mecham", page)

#music(98,"Rest Hymn",page)
#specialmusic("Elder Caleb Short and Brother Jared Draney",page, title= "Joseph Smith's First Prayer")
#specialmusic("Walk-Up Choir",page, title="Christ The Lord is Risen Today")
speaker("Reed Bradford", page)

speaker("President Travis Bird", page)
#speaker("Speaker 3", page)


### Closing hymn

In [27]:
music(34,"Closing Hymn",page)

## End matter - thought, announcements, links

In [28]:
# Spiritual thought
quote = ('No one of us is less treasured or cherished of God than another. I testify that He loves each of us insecurities, anxieties, self-image, and all.')
author = 'Elder Jeffrey R. Holland Of the Quorum of the Twelve Apostles April 2002'
thought(quote, author, page)

# Make a list of announcements, then post to the page
txtlist = (
        "Ward Temple Night - Saturday, August 5th, 6pm",
        "Just Serve : Houston South Stake - Back to School Clothing and Supply Drive for Fort Bend ISD & Lamar CISD",
                        "July 1st thru 30th Contact MeliaBoyer@justserve.org",
        "July 23rd, 2023 - FSY Wrap-Up Fireside  @ Lexington Building 7:00 PM to 9:00 PM",
        "Volley Ball Wednesdays - Come join us on Wednesdays at 9PM and have fun!!!",
        "Houston Texas South Stake : We are collecting information from those who may be interested in participating in one or both of the following choir opportunities:",
                        "9/11 Day of Service and Remembrance Ceremony Community Choir at the Houston National Cemetery on Saturday, September 9",
        "Send announcements to bvl2clerk@gmail.com"
           )
announcements(txtlist, page)

# Cleaning Assignments

In [29]:
sats = nextsaturday(4)
cleaninglist = []
for sat in sats:
    cleaninglist.append(sat + ": " + roster[sat])
custom("Cleaning Assignments",cleaninglist, None, page)

### Preparedness Corner - under development

In [30]:
#txtlist = (
#    'Make sure you have water: one gallon per person per day. A family of four would need 12 gallons for 3 days.',
#    ' '
#    )
#linklist = {
#    ("Learn more online","https://www.churchofjesuschrist.org/study/manual/gospel-topics/food-storage?lang=eng")
#}
#custom("Preparedness Corner", txtlist,linklist, page)

In [31]:
# Make a list of links, then post
linklist = (
        ("Let the missionaries know when you are available to help teach","https://docs.google.com/forms/d/1cVhEINFFESai0MLbgBA8euTyiMQsM8R1Sur_EQsyUrw/edit?usp=drive_web"),
        ("9/11 Day of Service and Remembrance Ceremony Community Choir at the Houston National Cemetery on Saturday, September 9", "https://docs.google.com/forms/d/e/1FAIpQLSd_FPE2TJCaNZ3Md3y5dTSKCmpAU7M5w6_tiLS_J28h5Ufq9Q/viewform"),
        ("Sign up to feed the missionaries","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-dinner2"),
        ("Sign up to prepare a meal with Fort Bend Family Promise","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-helping4#"),
        ("Help at secondmile.org","https://secondmile.org"), 
        )
links(linklist, page)
credit("Pete Slater and Andy Chen", page)

# Write out the file to Index.html, which must be posted online

In [32]:
print (page)# -*- coding: utf-8 -*

original_stdout = sys.stdout
with open('Index.html', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(page)
    sys.stdout = original_stdout # Reset the standard output to its original value
"""
End of script
"""

<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
<html lang="en">
<head>
</head>
<body>
<br />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<h1 align="center">Sugar Land Second Ward</h1>
<h2 align="center">Sacrament Meeting</h2>
<h2 align="center">July 23rd, 2023</h2>
<p align="center"><img style="max-width:50%;height:auto;" src="https://assets.ldscdn.org/e4/40/e440b004051c60f0e7b85e8af9b5bfbca156cf2d/houston_texas_mormon_temple.jpeg" /></p>
<p align="center"><b>Presiding:</b> President Travis Bird</p>
<p align="center"><b>Conducting:</b> Brother Jared Draney</p>
<h2 align="center">Opening Hymn</h2>
<p align="center"><a href="https://www.churchofjesuschrist.org/music/library/hymns/come-come-ye-saints?lang=eng&amp;#39">#30, Come, Come, Ye Saints</a></p>
<h2 align="center">Sacrament Hymn</h2>
<p align="center"><a href="https://www.churchofjesuschrist.org/music/library/hymns/tis-sweet-to-sing-the-matchless-love-meredith?lang=eng&amp;#39">#176, Ti

'\nEnd of script\n'